In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#load data item_proprties_part1 and item_proprties_part2 and concatenate them into one dataset
df_item_properties_part1 = pd.read_csv('item_properties_part1.1.csv', index_col=None)
df_item_properties_part2 = pd.read_csv('item_properties_part2.csv', index_col=None)
df_item_properties = pd.concat([df_item_properties_part1, df_item_properties_part2], ignore_index=True)
df_item_properties.head()

,timestamp,itemid,property,value
0,1.435460e+12,460429,categoryid,1338
1,1.441510e+12,206783,888,1116713 960601 n277.200
2,1.439090e+12,395014,400,n552.000 639502 n720.000 424566
3,1.431230e+12,59481,790,n15360.000
4,1.431830e+12,156781,917,828513


In [3]:
# load behaviour events dataset
df_events = pd.read_csv('events.csv', index_col=None)
df_events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1.433220e+12,257597,view,355908,NaN
1,1.433220e+12,992329,view,248676,NaN
2,1.433220e+12,111016,view,318965,NaN
3,1.433220e+12,483717,view,253185,NaN
4,1.433220e+12,951259,view,367447,NaN


In [4]:
# load category tree dataset
df_category_tree = pd.read_csv('category_tree.csv')
df_category_tree.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


In [5]:
#shape of the data sets
df_item_properties.shape

(2097150, 4)

In [6]:
df_events.shape

(1048575, 5)

In [7]:
df_category_tree.shape

(1669, 2)

In [8]:
#data types of the datasets
df_item_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097150 entries, 0 to 2097149
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  float64
 1   itemid     int64  
 2   property   object 
 3   value      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 64.0+ MB


In [9]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   timestamp      1048575 non-null  float64
 1   visitorid      1048575 non-null  int64  
 2   event          1048575 non-null  object 
 3   itemid         1048575 non-null  int64  
 4   transactionid  8654 non-null     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 40.0+ MB


In [10]:
df_category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


### Merge the 3 datasets

In [11]:
# merging df_events and df_item_prperties

In [12]:
df_events_props = pd.merge(
    df_events, 
    df_item_properties, 
    on='itemid', 
    how='left'
)

In [13]:
# merging category tree with the fist merge

In [14]:
# Filter only category rows
df_categories = df_item_properties[df_item_properties['property'] == 'categoryid']
df_categories = df_categories[['itemid', 'value']].rename(columns={'value': 'categoryid'})

In [15]:
# Convert categoryid to numeric for joining
df_categories['categoryid'] = pd.to_numeric(df_categories['categoryid'], errors='coerce')

In [16]:
# Merge category IDs into merged events/properties
df_events_props_categories = pd.merge(
    df_events_props,
    df_categories[['itemid', 'categoryid']],
    on='itemid',
    how='left'
)

In [17]:
# Merge with category tree
df_events_props_tree = pd.merge(
    df_events_props_categories,
    df_category_tree,
    on='categoryid',
    how='left'
)

In [18]:
# renaming the final dataset to df
df = df_events_props_tree
df.head()

,timestamp_x,visitorid,event,itemid,transactionid,timestamp_y,property,value,categoryid,parentid
0,1.433220e+12,257597,view,355908,NaN,1.431830e+12,159,519769,NaN,NaN
1,1.433220e+12,257597,view,355908,NaN,1.431830e+12,available,1,NaN,NaN
2,1.433220e+12,257597,view,355908,NaN,1.440900e+12,available,1,NaN,NaN
3,1.433220e+12,257597,view,355908,NaN,1.436070e+12,available,1,NaN,NaN
4,1.433220e+12,257597,view,355908,NaN,1.437880e+12,available,1,NaN,NaN


In [19]:
df.shape

(8056483, 10)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8056483 entries, 0 to 8056482
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp_x    float64
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
 5   timestamp_y    float64
 6   property       object 
 7   value          object 
 8   categoryid     float64
 9   parentid       float64
dtypes: float64(5), int64(2), object(3)
memory usage: 676.1+ MB


### Convert timestamps to human readable format

In [21]:
df['timestamp_x'] = pd.to_datetime(df['timestamp_x'] / 1000, unit='s')
df['timestamp_y'] = pd.to_datetime(df['timestamp_y'] / 1000, unit='s')
df.head()

,timestamp_x,visitorid,event,itemid,transactionid,timestamp_y,property,value,categoryid,parentid
0,2015-06-02 04:40:00,257597,view,355908,NaN,2015-05-17 02:33:20,159,519769,NaN,NaN
1,2015-06-02 04:40:00,257597,view,355908,NaN,2015-05-17 02:33:20,available,1,NaN,NaN
2,2015-06-02 04:40:00,257597,view,355908,NaN,2015-08-30 02:00:00,available,1,NaN,NaN
3,2015-06-02 04:40:00,257597,view,355908,NaN,2015-07-05 04:20:00,available,1,NaN,NaN
4,2015-06-02 04:40:00,257597,view,355908,NaN,2015-07-26 03:06:40,available,1,NaN,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8056483 entries, 0 to 8056482
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   timestamp_x    datetime64[ns]
 1   visitorid      int64         
 2   event          object        
 3   itemid         int64         
 4   transactionid  float64       
 5   timestamp_y    datetime64[ns]
 6   property       object        
 7   value          object        
 8   categoryid     float64       
 9   parentid       float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(3)
memory usage: 676.1+ MB


In [23]:
df.isna().sum()

timestamp_x            0
visitorid              0
event                  0
itemid                 0
transactionid    7983312
timestamp_y       120673
property          120673
value             120673
categoryid       5755095
parentid         5755270
dtype: int64

In [24]:
def clean_item_properties(df):
    """
    Cleans merged dataset by:
    1. Forward filling missing property/value within each itemid.
    """
    # Forward fill property and value within each item
    df[['property', 'value']] = (
        df.groupby('itemid')[['property', 'value']]
        .ffill()
        .bfill()  # also backfill if the first rows are NaN
    )    
    
    return df
df = clean_item_properties(df)

In [25]:
df.head()

,timestamp_x,visitorid,event,itemid,transactionid,timestamp_y,property,value,categoryid,parentid
0,2015-06-02 04:40:00,257597,view,355908,NaN,2015-05-17 02:33:20,159,519769,NaN,NaN
1,2015-06-02 04:40:00,257597,view,355908,NaN,2015-05-17 02:33:20,available,1,NaN,NaN
2,2015-06-02 04:40:00,257597,view,355908,NaN,2015-08-30 02:00:00,available,1,NaN,NaN
3,2015-06-02 04:40:00,257597,view,355908,NaN,2015-07-05 04:20:00,available,1,NaN,NaN
4,2015-06-02 04:40:00,257597,view,355908,NaN,2015-07-26 03:06:40,available,1,NaN,NaN


In [26]:
df.isna().sum()

timestamp_x            0
visitorid              0
event                  0
itemid                 0
transactionid    7983312
timestamp_y       120673
property               0
value                  0
categoryid       5755095
parentid         5755270
dtype: int64